In [103]:
import torch
import torch.nn as nn
from torch.autograd import Variable as V
import torch.nn.functional as F
from  sklearn.preprocessing import MinMaxScaler
import numpy as np
from sklearn.ensemble import IsolationForest
import pandas as pd
from torch.utils.data import DataLoader,Dataset,TensorDataset
import torch.optim as optim
import torch.nn.init as init
import numpy
from sklearn.model_selection import train_test_split

In [51]:
# load in datasets
RNAseq=pd.read_csv("/Users/vivit/CC/data_rnaSeq.csv")
train_clinical=pd.read_csv("/Users/vivit/CC/train_clinical_B73.csv")
test_clinical=pd.read_csv("/Users/vivit/CC/test_clinical_B73.csv")

In [52]:
train_idx=train_clinical[['attrib_name']]
test_idx=test_clinical[['attrib_name']]

In [53]:
train_idx

,attrib_name
0,TCGA.BH.A5J0
1,TCGA.AC.A3QQ
2,TCGA.C8.A3M8
3,TCGA.AC.A5EH
4,TCGA.AC.A2FE
...,...
426,TCGA.AC.A2QI
427,TCGA.D8.A1XC
428,TCGA.E2.A1IJ
429,TCGA.E2.A14U


In [54]:
RNAseq

,Unnamed: 0,A1BG,A1CF,A2BP1,A2LD1,A2ML1,A2M,A4GALT,A4GNT,AAA1,...,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,psiTPTE22,tAKR
0,TCGA.3C.AAAU,7.6300,0.0000,0.0000,6.6999,1.2501,12.5017,6.1136,3.2655,0.4273,...,7.0289,9.9784,10.6965,8.0196,10.2400,11.7765,10.8887,10.2064,1.4454,0.0000
1,TCGA.3C.AALI,7.8971,0.0000,0.0000,6.1672,2.4196,12.8866,7.3101,0.6265,0.0000,...,5.9264,8.8125,10.3924,7.6401,9.2398,12.4267,10.3659,8.6715,9.8573,0.0000
2,TCGA.3C.AALJ,8.7287,0.9310,0.0000,7.3422,0.0000,13.1101,9.1671,0.0000,0.0000,...,5.1842,9.0634,9.5884,8.3786,9.0585,12.4146,9.8825,8.9958,5.1842,0.0000
3,TCGA.3C.AALK,7.5851,0.0000,0.0000,5.9888,1.4087,13.4201,8.9870,0.0000,0.0000,...,5.8080,8.7772,9.7564,7.4629,9.2488,12.4748,9.6113,9.4551,6.0792,0.0000
4,TCGA.4H.AAAK,8.0762,0.5115,2.2720,7.2796,2.1389,13.2268,8.4226,0.5115,0.5115,...,5.6420,8.7322,10.0373,3.9110,9.6013,11.9811,9.7020,9.7858,7.5564,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611,TCGA.WT.AB44,8.8835,0.0000,0.0000,5.9711,2.9100,12.4015,8.4828,0.8596,0.0000,...,4.6687,7.9339,9.5961,6.6374,8.3047,13.4171,9.6468,8.7947,9.5036,0.0000
612,TCGA.XX.A899,7.8088,0.0000,0.4789,7.0469,0.4789,14.3127,8.9845,0.8379,0.0000,...,6.1672,9.3325,10.3078,4.4764,9.4288,12.3217,10.9180,9.3903,6.6355,0.0000
613,TCGA.XX.A89A,8.0008,1.7492,1.2733,6.3386,5.8263,14.0614,9.2679,1.9389,0.9595,...,5.6338,8.4200,10.4268,7.3697,9.5465,12.4196,10.4903,9.8989,7.8841,0.9595
614,TCGA.Z7.A8R5,8.7831,0.0000,0.6756,6.3628,1.7610,15.1690,8.8372,0.0000,0.0000,...,5.0555,7.9628,9.6388,5.7270,8.9856,12.7049,9.5611,9.5543,7.9065,0.0000


In [55]:
RNAseq.shape

(616, 20156)

In [56]:
trainset = RNAseq.merge(train_idx,left_on="Unnamed: 0",right_on="attrib_name",how="inner")
testset = RNAseq.merge(test_idx,left_on="Unnamed: 0",right_on="attrib_name",how="inner")

In [83]:
trainset

,Unnamed: 0,A1BG,A1CF,A2BP1,A2LD1,A2ML1,A2M,A4GALT,A4GNT,AAA1,...,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,psiTPTE22,tAKR,attrib_name
0,TCGA.3C.AAAU,7.6300,0.0000,0.0000,6.6999,1.2501,12.5017,6.1136,3.2655,0.4273,...,9.9784,10.6965,8.0196,10.2400,11.7765,10.8887,10.2064,1.4454,0.0000,TCGA.3C.AAAU
1,TCGA.3C.AALI,7.8971,0.0000,0.0000,6.1672,2.4196,12.8866,7.3101,0.6265,0.0000,...,8.8125,10.3924,7.6401,9.2398,12.4267,10.3659,8.6715,9.8573,0.0000,TCGA.3C.AALI
2,TCGA.3C.AALJ,8.7287,0.9310,0.0000,7.3422,0.0000,13.1101,9.1671,0.0000,0.0000,...,9.0634,9.5884,8.3786,9.0585,12.4146,9.8825,8.9958,5.1842,0.0000,TCGA.3C.AALJ
3,TCGA.3C.AALK,7.5851,0.0000,0.0000,5.9888,1.4087,13.4201,8.9870,0.0000,0.0000,...,8.7772,9.7564,7.4629,9.2488,12.4748,9.6113,9.4551,6.0792,0.0000,TCGA.3C.AALK
4,TCGA.4H.AAAK,8.0762,0.5115,2.2720,7.2796,2.1389,13.2268,8.4226,0.5115,0.5115,...,8.7322,10.0373,3.9110,9.6013,11.9811,9.7020,9.7858,7.5564,0.0000,TCGA.4H.AAAK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426,TCGA.WT.AB41,6.9166,0.0000,2.5190,8.2903,0.0000,13.2261,10.9761,0.0000,0.0000,...,7.8086,8.8430,5.5708,7.8633,12.7050,9.1276,8.1703,4.8773,0.0000,TCGA.WT.AB41
427,TCGA.XX.A899,7.8088,0.0000,0.4789,7.0469,0.4789,14.3127,8.9845,0.8379,0.0000,...,9.3325,10.3078,4.4764,9.4288,12.3217,10.9180,9.3903,6.6355,0.0000,TCGA.XX.A899
428,TCGA.XX.A89A,8.0008,1.7492,1.2733,6.3386,5.8263,14.0614,9.2679,1.9389,0.9595,...,8.4200,10.4268,7.3697,9.5465,12.4196,10.4903,9.8989,7.8841,0.9595,TCGA.XX.A89A
429,TCGA.Z7.A8R5,8.7831,0.0000,0.6756,6.3628,1.7610,15.1690,8.8372,0.0000,0.0000,...,7.9628,9.6388,5.7270,8.9856,12.7049,9.5611,9.5543,7.9065,0.0000,TCGA.Z7.A8R5


In [57]:
trainset1=trainset.drop("Unnamed: 0",axis=1)
testset1=testset.drop("Unnamed: 0",axis=1)
trainset2=trainset1.drop("attrib_name",axis=1)
testset2=testset1.drop("attrib_name",axis=1)

In [58]:
trainset3=trainset2.to_numpy()
testset3=testset2.to_numpy()

In [96]:
test_idx_73=testset[["Unnamed: 0"]]

In [97]:
test_idx_73

,Unnamed: 0
0,TCGA.5L.AAT0
1,TCGA.A1.A0SF
2,TCGA.A1.A0SI
3,TCGA.A1.A0SQ
4,TCGA.A2.A0CO
...,...
180,TCGA.S3.AA12
181,TCGA.S3.AA17
182,TCGA.UL.AAZ6
183,TCGA.W8.A86G


In [98]:
test_clinical73=test_clinical.merge(test_idx_73,left_on="attrib_name",right_on="Unnamed: 0",how="inner")

In [99]:
test_clinical73=test_clinical73.drop("Unnamed: 0_x",axis=1)

In [100]:
test_clinical73

,attrib_name,years_to_birth,Tumor_purity,pathologic_stage,pathology_T_stage,pathology_N_stage,pathology_M_stage,histological_type,number_of_lymph_nodes,PAM50,...,HER2.Status,gender,radiation_therapy,race,ethnicity,Median_overall_survival,overall_survival,status,overallsurvival,Unnamed: 0_y
0,TCGA.A7.A6VY,48.0,0.5982,stageii,t2,n1,m0,infiltratingductalcarcinoma,2.0,NaN,...,NaN,female,yes,blackorafricanamerican,nothispanicorlatino,0.0,266.0,0.0,"266,0",TCGA.A7.A6VY
1,TCGA.A1.A0SI,52.0,0.6438,stageii,t2,n1,m0,infiltratingductalcarcinoma,1.0,LumB,...,NaN,female,NaN,white,nothispanicorlatino,0.0,635.0,0.0,"635,0",TCGA.A1.A0SI
2,TCGA.EW.A1PF,50.0,0.6464,stagei,t1,n0,m0,infiltratingductalcarcinoma,0.0,LumA,...,NaN,female,no,white,hispanicorlatino,0.0,439.0,0.0,"439,0",TCGA.EW.A1PF
3,TCGA.GM.A2DK,58.0,0.6851,stagei,t1,n0,m0,infiltratingductalcarcinoma,0.0,LumA,...,NaN,female,yes,white,nothispanicorlatino,0.0,2645.0,0.0,"2645,0",TCGA.GM.A2DK
4,TCGA.BH.A1FU,44.0,0.6121,stagei,t1,n0,m0,infiltratingductalcarcinoma,0.0,NaN,...,NaN,female,NaN,white,nothispanicorlatino,0.0,1688.0,1.0,"1688,1",TCGA.BH.A1FU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,TCGA.A2.A259,70.0,0.6159,stagei,t1,n0,m0,infiltratingductalcarcinoma,0.0,LumA,...,NaN,female,no,blackorafricanamerican,nothispanicorlatino,0.0,1596.0,0.0,"1596,0",TCGA.A2.A259
181,TCGA.LL.A5YP,49.0,0.6746,stageii,t2,n1,m0,infiltratingductalcarcinoma,2.0,Basal,...,NaN,female,yes,blackorafricanamerican,nothispanicorlatino,0.0,450.0,0.0,"450,0",TCGA.LL.A5YP
182,TCGA.BH.A1FH,47.0,0.6881,stageiv,t2,n1,m1,infiltratingductalcarcinoma,8.0,LumA,...,NaN,female,NaN,white,nothispanicorlatino,0.0,1034.0,1.0,"1034,1",TCGA.BH.A1FH
183,TCGA.E9.A54X,85.0,0.8966,stagei,t1,n0,m0,mucinouscarcinoma,0.0,LumB,...,NaN,female,no,white,nothispanicorlatino,0.0,727.0,0.0,"727,0",TCGA.E9.A54X


In [101]:
test_clinical73=test_clinical73.drop("Unnamed: 0_y",axis=1)

In [102]:
test_clinical73.to_csv("/Users/vivit/CC/test_clinical_B73.csv")

In [59]:
# standardize inputs
MMScaler=MinMaxScaler()
train_scaler=MMScaler.fit_transform(trainset3)
test_scaler=MMScaler.fit_transform(testset3)

In [60]:
train_torch = torch.FloatTensor(train_scaler)
train_loader = torch.utils.data.DataLoader(train_torch, batch_size = 128, shuffle=True)
test_torch = torch.FloatTensor(test_scaler)
test_loader = torch.utils.data.DataLoader(test_torch, batch_size = 128, shuffle=False)

In [61]:
# define parameters
EPOCH=10
lr=0.0001

In [62]:
train_torch.shape

torch.Size([431, 20155])

In [63]:
test_torch.shape

torch.Size([185, 20155])

In [64]:
class Autoencoder512(nn.Module):
    def __init__(self):
        super(Autoencoder512, self).__init__()
        # encoder
        self.encoder=nn.Sequential(
            nn.Linear(in_features=20155, out_features=2048),
            nn.ReLU(),
            nn.Linear(in_features=2048, out_features=1024),
            nn.ReLU(),
            nn.Linear(in_features=1024, out_features=512)  
            
        )
        self.decoder=nn.Sequential(
            nn.Linear(in_features=512, out_features=1024),
            nn.ReLU(),
            nn.Linear(in_features=1024, out_features=2048),
            nn.ReLU(),
            nn.Linear(in_features=2048, out_features=20155)
        )
 
    def forward(self, x):
        encoded=self.encoder(x)
        decoded=self.decoder(encoded)
        return decoded
    
    
model512 = Autoencoder512()
if torch.cuda.is_available():
    model512.cuda()
    
model512.train(mode=True)
# define optimizer
optimizer512 = torch.optim.Adam(model512.parameters(), lr=lr)
criterion512=nn.MSELoss()


In [65]:
# train the datasets
for epoch in range(EPOCH):
    for batch_features in train_torch:
        output = model512(batch_features)
        loss = criterion512(output, batch_features)
        optimizer512.zero_grad()
        loss.backward()
        optimizer512.step()

    print('epoch [{}/{}], loss:{:.4f}'.format(epoch + 1,EPOCH,loss.data.numpy()))

epoch [1/10], loss:0.0170
epoch [2/10], loss:0.0158
epoch [3/10], loss:0.0173
epoch [4/10], loss:0.0154
epoch [5/10], loss:0.0145
epoch [6/10], loss:0.0140
epoch [7/10], loss:0.0143
epoch [8/10], loss:0.0183
epoch [9/10], loss:0.0138
epoch [10/10], loss:0.0136


In [73]:
feature=model512.encoder(train_torch)
feature

tensor([[-0.1157, -0.1136,  0.4448,  ..., -0.4854, -0.6243,  0.0336],
        [-0.2169, -0.2199,  0.1744,  ..., -0.4718, -0.4241,  0.0441],
        [-0.1700, -0.1489,  0.1683,  ..., -0.4276, -0.4835,  0.0593],
        ...,
        [-0.3500, -0.0438,  0.0850,  ..., -0.5056, -0.5599,  0.1732],
        [-0.1541,  0.2141, -0.0206,  ..., -0.2818, -0.5733,  0.1673],
        [-0.1186, -0.0888,  0.2766,  ..., -0.4272, -0.5638,  0.0471]],
       grad_fn=<AddmmBackward>)

In [74]:
a = feature.detach().numpy()
print(a)

[[-0.11567852 -0.11362448  0.44483757 ... -0.48535365 -0.6243105
   0.03356799]
 [-0.21687448 -0.21986358  0.17441802 ... -0.47176436 -0.42413723
   0.04410264]
 [-0.16999635 -0.1488511   0.16834886 ... -0.4276497  -0.48354036
   0.0593442 ]
 ...
 [-0.3499763  -0.04381548  0.08500226 ... -0.50562406 -0.5599162
   0.17317781]
 [-0.15409485  0.21406634 -0.02063711 ... -0.28182396 -0.5732734
   0.16732481]
 [-0.1185642  -0.08884819  0.27660894 ... -0.42723474 -0.5638191
   0.04710686]]


In [75]:
a.shape

(431, 512)

In [76]:
np.savetxt("/Users/vivit/CC/RNAseq AE results for Breast Cancer 73.csv",a,delimiter=',')

In [77]:
torch.save(model512, '/Users/vivit/CC/AE model for RNAseq 73.pkl')

In [78]:
f=model512.encoder(test_torch)
f

tensor([[-0.2512,  0.0282,  0.1435,  ..., -0.4461, -0.6070,  0.1497],
        [-0.2359, -0.0379,  0.2054,  ..., -0.4758, -0.5698,  0.1233],
        [-0.3165, -0.2291,  0.1124,  ..., -0.5304, -0.3440,  0.1295],
        ...,
        [ 0.0008, -0.0535,  0.2434,  ..., -0.3382, -0.5373,  0.0500],
        [-0.2239,  0.1469,  0.2016,  ..., -0.4274, -0.7041,  0.1503],
        [-0.1345,  0.1822,  0.0554,  ..., -0.3086, -0.5947,  0.1602]],
       grad_fn=<AddmmBackward>)

In [79]:
b = f.detach().numpy()
b.shape

(185, 512)

In [80]:
np.savetxt("/Users/vivit/CC/RNAseq AE test results for Breast Cancer 73.csv",b,delimiter=',')

In [66]:
class Autoencoder256(nn.Module):
    def __init__(self):
        super(Autoencoder256, self).__init__()
        # encoder
        self.encoder=nn.Sequential(
            nn.Linear(in_features=20155, out_features=1024),
            nn.ReLU(),
            nn.Linear(in_features=1024, out_features=512),
            nn.ReLU(),
            nn.Linear(in_features=512, out_features=256),
        )
        self.decoder=nn.Sequential(
            nn.Linear(in_features=256, out_features=512),
            nn.ReLU(),
            nn.Linear(in_features=512, out_features=1024),
            nn.ReLU(),
            nn.Linear(in_features=1024, out_features=20155),
        )
 
    def forward(self, x):
        encoded=self.encoder(x)
        decoded=self.decoder(encoded)
        return encoded,decoded
    
    
model256 = Autoencoder256()
if torch.cuda.is_available():
    model256.cuda()
    
model256.train(mode=True)
# define optimizer
optimizer256 = torch.optim.Adam(model256.parameters(), lr=lr)
criterion256=nn.MSELoss()

In [67]:
# train the datasets
for epoch in range(EPOCH):
    for batch_features in train_torch:
        _,output = model256(batch_features)
        loss = criterion256(output, batch_features)    
        optimizer256.zero_grad()
        loss.backward()
        optimizer256.step()

    print('epoch [{}/{}], loss:{:.4f}'.format(epoch + 1,EPOCH,loss.data.numpy()))

epoch [1/10], loss:0.0170
epoch [2/10], loss:0.0161
epoch [3/10], loss:0.0159
epoch [4/10], loss:0.0139
epoch [5/10], loss:0.0137
epoch [6/10], loss:0.0146
epoch [7/10], loss:0.0137
epoch [8/10], loss:0.0136
epoch [9/10], loss:0.0136
epoch [10/10], loss:0.0144


In [68]:
class Autoencoder128(nn.Module):
    def __init__(self):
        super(Autoencoder128, self).__init__()
        # encoder
        self.encoder=nn.Sequential(
            nn.Linear(in_features=20155, out_features=512),
            nn.ReLU(),
            nn.Linear(in_features=512, out_features=256),
            nn.ReLU(),
            nn.Linear(in_features=256, out_features=128),
        )
        self.decoder=nn.Sequential(
            nn.Linear(in_features=128, out_features=256),
            nn.ReLU(),
            nn.Linear(in_features=256, out_features=512),
            nn.ReLU(),
            nn.Linear(in_features=512, out_features=20155),
        )
 
    def forward(self, x):
        encoded=self.encoder(x)
        decoded=self.decoder(encoded)
        return encoded,decoded
    
    
model128 = Autoencoder128()
if torch.cuda.is_available():
    model128.cuda()
    
model128.train(mode=True)
# define optimizer
optimizer128 = torch.optim.Adam(model128.parameters(), lr=lr)
criterion128=nn.MSELoss()

In [69]:
# train the datasets
for epoch in range(EPOCH):
    for batch_features in train_torch:
        _,output = model128(batch_features)
        loss = criterion128(output, batch_features)    
        optimizer128.zero_grad()
        loss.backward()
        optimizer128.step()

    print('epoch [{}/{}], loss:{:.4f}'.format(epoch + 1,EPOCH,loss.data.numpy()))

epoch [1/10], loss:0.0162
epoch [2/10], loss:0.0165
epoch [3/10], loss:0.0156
epoch [4/10], loss:0.0145
epoch [5/10], loss:0.0141
epoch [6/10], loss:0.0142
epoch [7/10], loss:0.0152
epoch [8/10], loss:0.0138
epoch [9/10], loss:0.0136
epoch [10/10], loss:0.0138


In [70]:
class Autoencoder64(nn.Module):
    def __init__(self):
        super(Autoencoder64, self).__init__()
        # encoder
        self.encoder=nn.Sequential(
            nn.Linear(in_features=20155, out_features=256),
            nn.ReLU(),
            nn.Linear(in_features=256, out_features=128),
            nn.ReLU(),
            nn.Linear(in_features=128, out_features=64),
        )
        self.decoder=nn.Sequential(
            nn.Linear(in_features=64, out_features=128),
            nn.ReLU(),
            nn.Linear(in_features=128, out_features=256),
            nn.ReLU(),
            nn.Linear(in_features=256, out_features=20155),
        )
 
    def forward(self, x):
        encoded=self.encoder(x)
        decoded=self.decoder(encoded)
        return encoded,decoded
    
    
model64 = Autoencoder64()
if torch.cuda.is_available():
    model64.cuda()
    
model64.train(mode=True)
# define optimizer
optimizer64 = torch.optim.Adam(model64.parameters(), lr=lr)
criterion64=nn.MSELoss()

In [71]:
# train the datasets
for epoch in range(EPOCH):
    for batch_features in train_torch:
        _,output = model64(batch_features)
        loss = criterion64(output, batch_features)    
        optimizer64.zero_grad()
        loss.backward()
        optimizer64.step()

    print('epoch [{}/{}], loss:{:.4f}'.format(epoch + 1,EPOCH,loss.data.numpy()))

epoch [1/10], loss:0.0163
epoch [2/10], loss:0.0172
epoch [3/10], loss:0.0164
epoch [4/10], loss:0.0169
epoch [5/10], loss:0.0167
epoch [6/10], loss:0.0160
epoch [7/10], loss:0.0156
epoch [8/10], loss:0.0147
epoch [9/10], loss:0.0140
epoch [10/10], loss:0.0144


In [46]:
from sklearn.model_selection import KFold,StratifiedKFold
KF=KFold(n_splits=10,shuffle=True,random_state=123)
print(KF)

KFold(n_splits=10, random_state=123, shuffle=True)


In [47]:
def train_epoch(model,device,dataloader,loss_fun,optimizer):
    train_loss=0.0
    model.train()
    for x in dataloader:
        x=x.to(device)
        optimizer.zero_grad()
        output = model(x)
        loss = loss_fun(output,x)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss=train_loss/len(dataloader)
    return train_loss
def valid_epoch(model,device,dataloader,loss_fun):
    valid_loss = 0.0
    model.eval()
    for x in dataloader:
        x=x.to(device)
        output = model(x)
        loss=loss_func(output,x)
        valid_loss+=loss.item()
    valid_loss+=valid_loss/len(dataloader)
    return valid_loss

In [72]:
from torch.utils.data import Dataset, DataLoader,TensorDataset,random_split,SubsetRandomSampler, ConcatDataset 
criterion=nn.MSELoss()
K_valid_loss=[]
for fold, (train_idx,val_idx) in enumerate(KF.split(np.arange(len(test_torch)))):
    print('Fold {}'.format(fold + 1))
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(val_idx)
    train_loader = torch.utils.data.DataLoader(train_torch,batch_size=512,sampler=train_sampler)
    test_loader = torch.utils.data.DataLoader(test_torch,batch_size=512,sampler=test_sampler)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = Autoencoder512()
    model.to(device)
    loss_func=nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    history = {'train_loss': [], 'test_loss': []}
    for epoch in range(EPOCH):
        train_loss=train_epoch(model,device,train_loader,criterion,optimizer)
        test_loss=valid_epoch(model,device,test_loader,criterion)
        print("Epoch:{}/{} AVG Training Loss:{:.3f} AVG Test Loss:{:.3f} ".format(epoch + 1,EPOCH,train_loss, test_loss))

Fold 1
Epoch:1/10 AVG Training Loss:0.230 AVG Test Loss:1.706 
Epoch:2/10 AVG Training Loss:0.812 AVG Test Loss:0.427 
Epoch:3/10 AVG Training Loss:0.202 AVG Test Loss:0.297 
Epoch:4/10 AVG Training Loss:0.139 AVG Test Loss:0.518 
Epoch:5/10 AVG Training Loss:0.240 AVG Test Loss:0.147 
Epoch:6/10 AVG Training Loss:0.065 AVG Test Loss:0.229 
Epoch:7/10 AVG Training Loss:0.105 AVG Test Loss:0.219 
Epoch:8/10 AVG Training Loss:0.100 AVG Test Loss:0.156 
Epoch:9/10 AVG Training Loss:0.069 AVG Test Loss:0.162 
Epoch:10/10 AVG Training Loss:0.068 AVG Test Loss:0.159 
Fold 2
Epoch:1/10 AVG Training Loss:0.230 AVG Test Loss:2.328 
Epoch:2/10 AVG Training Loss:1.152 AVG Test Loss:0.418 
Epoch:3/10 AVG Training Loss:0.204 AVG Test Loss:0.324 
Epoch:4/10 AVG Training Loss:0.157 AVG Test Loss:0.238 
Epoch:5/10 AVG Training Loss:0.114 AVG Test Loss:0.101 
Epoch:6/10 AVG Training Loss:0.046 AVG Test Loss:0.093 
Epoch:7/10 AVG Training Loss:0.041 AVG Test Loss:0.103 
Epoch:8/10 AVG Training Loss:0.04